Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = ""
COLLABORATORS = ""

---

In [3]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [5]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [6]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [7]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).geom_type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).geom_type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [8]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [9]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

,GEOID20,District
0,34033020001,2
1,34033001501,2
2,34033042009,2
3,34033000703,2
4,34033042001,2
...,...,...
6356,34039045302,7
6357,34039045303,7
6358,34039045404,7
6359,34039045801,7


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [10]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [11]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [12]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [13]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(i, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [14]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

D1 PP : 0.41768102211569347
D1 BR : 0.45075813446417157
D2 PP : 0.2632665176502347
D2 BR : 0.38278056561756263
D3 PP : 0.2280937682959879
D3 BR : 0.38134920642809134
D4 PP : 0.24812480573284196
D4 BR : 0.5390173747196018
D5 PP : 0.2410116694999733
D5 BR : 0.36320426268176653
D6 PP : 0.14677124653853732
D6 BR : 0.32853496486220907
D7 PP : 0.20246375771704353
D7 BR : 0.44049249082841035
D8 PP : 0.11227347882175574
D8 BR : 0.36670629634952656
D9 PP : 0.1683197710884751
D9 BR : 0.29705227593212374
D10 PP : 0.12061263370064262
D10 BR : 0.34528827672774703
D11 PP : 0.22236600778446886
D11 BR : 0.5557086439792166
D12 PP : 0.1620092442171186
D12 BR : 0.38520439164401626


In [15]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

{1: 775340, 2: 778354, 3: 778489, 4: 767834, 5: 774454, 6: 778516, 7: 785173, 8: 800074, 9: 766863, 10: 746178, 11: 769523, 12: 768196}


# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West. 
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [16]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).geom_type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

In [17]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    

D1 PP : 0.24444322502174795
D1 BR : 0.39921574154934536
D2 PP : 0.21732375139633123
D2 BR : 0.3033805826444085
D3 PP : 0.3012927570664991
D3 BR : 0.7281613087250183
D4 PP : 0.3019375759906228
D4 BR : 0.6388831361756292
D5 PP : 0.29897783517755583
D5 BR : 0.5136294328843819
D6 PP : 0.22596255307162377
D6 BR : 0.5362987383285662
D7 PP : 0.21866681223528045
D7 BR : 0.4337413755961903
D8 PP : 0.3513739185552307
D8 BR : 0.82797734690796
D9 PP : 0.37261194943216025
D9 BR : 0.7269867982048943
D10 PP : 0.28764401412152446
D10 BR : 0.6900841778845721
D11 PP : 0.34638282192061104
D11 BR : 0.47137706735305646
D12 PP : 0.318631303145351
D12 BR : 0.5828294919378438


In [18]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


{1: 80406, 2: 22492, 3: 317218, 4: 274411, 5: 1124427, 6: 563593, 7: 244052, 8: 1471840, 9: 230117, 10: 3848094, 11: 61026, 12: 1051318}


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

In [36]:
import random
import math
from collections import deque
import pandas as pd
import numpy as np
import ast
import networkx as nx
from shapely.geometry import Polygon, shape, MultiPolygon
import shapely.ops

# useful data extraction to get population of precinct, the number of democrats and republicans
K = 12
POP_COL = 'Total_2020_Total'
DEM_COL = 'Dem_2020_Pres'
REP_COL = 'Rep_2020_Pres'

# used for spread of seed precincts 
BFS_SPREAD_DISTANCE = 7

total_population = nj_precinct_data['Total_2020_Total'].sum()
target = total_population / K
ideal_population = target
MIN_POP = ideal_population * 0.9550
MAX_POP = ideal_population * 1.0450



# used to load in contiogus precincts, by getting adjacent precincts from data provided
print("Building precinct contiguity graphs (G and adj)...")
G = nx.Graph()
all_ids = nj_precinct_data['GEOID20'].tolist()
G.add_nodes_from(all_ids)

nj_contiguity.columns = ['Precinct','Neighbors']
for index, row in nj_contiguity.iterrows():
    precinct = row['Precinct']
    try:
        neighbors = ast.literal_eval(row['Neighbors'])
        for neighbor in neighbors:
            if neighbor in all_ids:
                G.add_edge(precinct, neighbor)
    except Exception:
        pass

adj = {node: list(G.neighbors(node)) for node in G.nodes()}

row_by_id = nj_precinct_data.set_index('GEOID20').to_dict('index')




if 'nj_precinct_boundaries' not in locals():
    print("error: 'nj_precinct_boundaries' missing!")
    nj_precinct_boundaries = {}

# this breaks the NJ map into three regions top left, top right, and bottom. Useful to select the seed precicnts
# this uses the precincts 
MID_LON = -74.72
MID_LAT = (38.92 + 41.35) / 2


#Different values here will result in different outcomes for fair/unfair map
SEED_COUNTS = {
    'TL': 3,
    'TR': 4,
    'Bottom': 5
}


precinct_zones = {'TL': [], 'TR': [], 'Bottom': []}


for geoid in all_ids:
    if geoid not in nj_precinct_boundaries:
        continue
    
    try:
        geom = shape(nj_precinct_boundaries[geoid])
        centroid = geom.centroid

        if centroid.y > MID_LAT:
            if centroid.x < MID_LON:
                precinct_zones['TL'].append(geoid)
            else:
                precinct_zones['TR'].append(geoid)
        else:
            precinct_zones['Bottom'].append(geoid)

    except Exception:
        continue


# goal is to have precincts be decently far from eachother so they don't block one another, this random 
#precinct selection will result in different outcome for fair/unfair, sometimes 
# it could also result in unusual outcomes
def _select_seeds_bfs(geoid_list, num_seeds, avoid_set):
    selected = []
    shuffled = random.sample(geoid_list, len(geoid_list))
    for geoid in shuffled:
        if geoid in avoid_set:
            continue   
        selected.append(geoid)
        q = deque([(geoid, 0)])
        visited = {geoid}

        while q:
            node, dist = q.popleft()
            if dist >= BFS_SPREAD_DISTANCE:
                continue

            avoid_set.add(node)

            for nbr in adj.get(node, []):
                if nbr not in visited:
                    visited.add(nbr)
                    q.append((nbr, dist + 1))
        if len(selected) == num_seeds:
            break
            
    if len(selected) < num_seeds:
        print(f"WARNING: Could only pick {len(selected)}/{num_seeds} seeds in this zone.")

    return selected, avoid_set
    
seeds = []
avoid = set()


new_tl, avoid = _select_seeds_bfs(precinct_zones['TL'], SEED_COUNTS['TL'], avoid)
seeds.extend(new_tl)
new_tr, avoid = _select_seeds_bfs(precinct_zones['TR'], SEED_COUNTS['TR'], avoid)
seeds.extend(new_tr)
new_bot, avoid = _select_seeds_bfs(precinct_zones['Bottom'], SEED_COUNTS['Bottom'], avoid)
seeds.extend(new_bot)

if len(seeds) == K:
    print("seed precinct selection successful: ", seeds, " ", len(seeds))

Building precinct contiguity graphs (G and adj)...
seed precinct selection successful:  ['34019110008', '34021055N07', '34041080003', '34025125001', '34013055010', '34021050015', '34017030F05', '34005110003', '34007040308', '34007050003', '34005075203', '34011070103']   12


In [37]:

# given function, could have used this to make a more compact map, but at the cost of the algorithm being 
# extremely slow
def getDistrictShape(district_id, assignment, boundaries):
    if isinstance(assignment, dict):
        assignment = pd.DataFrame(assignment.items(), columns=['GEOID20', 'District'])

    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        try:
            geom = boundaries.get(i)
            if geom:
                s = shape(geom)
                if s.geom_type == 'Polygon':
                    precinct_shapes.append(s)
                elif s.geom_type == 'MultiPolygon':
                    precinct_shapes.append(s)
        except Exception:
            pass 
    
    if not precinct_shapes:
        return Polygon()
        
    return shapely.ops.unary_union(precinct_shapes)
    
# our goal was to use this given formula for compactness, but it took way too long to find the shape of a 
# particular district, so we scratched that idea
def pp_compactness(geom):
    if geom.area == 0:
        return 0.0
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def build_map_balanced(objective="fair", advantage="DEM", seed_ids=None):
    
    district_index = 0
    if seed_ids is None:
        seed_ids = seeds
    assigned = {}
    district_pop = [0]*K
    district_dem = [0]*K
    district_rep = [0]*K
    district_precincts = [1]*K
    frontiers = [set() for _ in range(K)]

    for d, gid in enumerate(seed_ids):
        assigned[gid] = d
        r = row_by_id[gid]
        district_pop[d] += int(r[POP_COL])
        district_dem[d] += int(r[DEM_COL])
        district_rep[d] += int(r[REP_COL])
        for nbr in adj[gid]:
            if nbr not in assigned:
                frontiers[d].add(nbr)

    unassigned = set(all_ids) - set(assigned.keys())
    upper5 = target * (1 + 0.05)
    lower5 = target * (1 - 0.05)

    # this shortest path tris to find the shortest path from one potential precinct canditate to the
    # center (or seed precinct), this is what we used to make it more compact
    # we believe this could have been better if we used the given compactness functions, but we wanted to 
    # make it less time consuming and we ended using this way to try to make comapct districts
    def shortest_path_lengths_to_frontiers(d):
        seed = seeds[d]
        lengths = {}
        for frontier in frontiers[d]:
            visited = {seed}
            q = deque([(seed, 0)])
            found = False
            while q:
                node, dist = q.popleft()
                if node == frontier:
                    lengths[frontier] = dist
                    found = True
                    break
                for nbr in adj.get(node, []):
                    if nbr not in visited:
                        visited.add(nbr)
                        q.append((nbr, dist + 1))
            if not found:
                lengths[frontier] = float('inf')
        return lengths

    # this function is to get the partisan hueristic where the unfair part uses an unfair ratio to give one party a 
    #better chance over another party
    def partisan_score(d, gid):
        r = row_by_id[gid]
        dem = int(r[DEM_COL])
        rep = int(r[REP_COL])
        total = dem + rep
        if total == 0: return 0.5 
        
        if objective == "unfair":
            my_share = (rep / total) if advantage == "REP" else (dem / total)
            
            dist_total = district_dem[d] + district_rep[d]
            if dist_total == 0:
                dist_share = 0.5
            else:
                dist_share = (district_rep[d] / dist_total) if advantage == "REP" else (district_dem[d] / dist_total)
            
            SAFE_LIMIT = 0.53
            LOST_LIMIT = 0.47

            if dist_share > SAFE_LIMIT:
                # Safe: Pack Opponent (Minimize my_share)
                return my_share
            elif dist_share < LOST_LIMIT:
             
                return my_share
            else:
            
                return 1.0 - my_share
        else:
            comp = 1 - abs(dem - rep) / max(dem + rep, 1)
            return -comp

    # this function is to get the potential expected population that is left for a district to collect within the range of 25
    # disclaimer: we choose 25 becuase that is a number that we thought was good, but we believe this could be altered 
    # to create a better outcome
    def bfs_unassigned_population(d, max_iter=25):
        visited = set()
        q = deque([(f, 0) for f in frontiers[d]])
        total_pop = 0
    
        while q:
            node, dist = q.popleft()
            if node in visited or dist > max_iter:
                continue
            visited.add(node)
    
            if node in unassigned:
                total_pop += int(row_by_id[node][POP_COL])
    
            for nbr in adj.get(node, []):
                if nbr not in visited:
                    q.append((nbr, dist + 1))
        
        return total_pop

    # This function's goal is to get the distance from current precinct to a nearby district to use it for 
    #the heuristic score

    def distance_to_other_districts(gid, d):
        visited = {gid}
        q = deque([(gid, 0)])
        while q:
            node, dist = q.popleft()
            for nbr in adj.get(node, []):
                if nbr in assigned and assigned[nbr] != d:
                    return dist + 1
                if nbr not in visited and nbr in unassigned:
                    visited.add(nbr)
                    q.append((nbr, dist + 1))
        return 50

    # iteravely go through the algorithm to assign districts based on current population of precinct/ expected 
    # population of the precincts, and heuristics (based on partisan divide, district to district distance, and 
    # precinct distance from center
    
    iter = 0
    while unassigned and iter < 6350:
        for d in range(K):
            frontiers[d] = {g for g in frontiers[d] if g in unassigned}

        candidates_d = [d for d in range(K) if district_pop[d] < target and frontiers[d]]
        if not candidates_d:
            candidates_d = [d for d in range(K) if frontiers[d]]
            if not candidates_d:
                break
        
        expected_pops = {d: bfs_unassigned_population(d) for d in candidates_d}
        
        district_scores = []
        for d in candidates_d:
            current_pop = district_pop[d]
            expected_pop = expected_pops[d]
            total_potential_pop = current_pop + expected_pop 
            district_scores.append((total_potential_pop, d)) 

        if district_scores:
            d = min(district_scores, key=lambda x: x[0])[1] 
        else:
            while True:
                d = district_index % K
                district_index += 1
                if d in candidates_d:
                    break
        
        best_gid, best_score = None, float('inf')
        paths_length = shortest_path_lengths_to_frontiers(d)

        for gid in list(frontiers[d]):
            if gid not in unassigned:
                continue
            r = row_by_id[gid]
            new_pop = district_pop[d] + int(r[POP_COL])
            
            pop_deviation_penalty = abs(new_pop - target) / target * 10
            bound_pen = 0
            if new_pop > upper5:
                bound_pen = (new_pop - upper5) / target * 1000 
            total_pop_penalty = pop_deviation_penalty + bound_pen
            
            proximity_dist = distance_to_other_districts(gid, d)
            if proximity_dist == 0:
                proximity_dist = 1

            # important part of the algorithm this is where we pick a precinct to put into a district based on a 
            # heuristic
            if objective == "unfair":
                score = ((total_pop_penalty + 1.5 * partisan_score(d, gid)) *
                        pow(paths_length.get(gid, 1),2))/pow(proximity_dist,0.5)
            else:
                score = ((total_pop_penalty + 1.5 * partisan_score(d, gid)) *
                        pow(paths_length.get(gid, 1),2))/ pow(proximity_dist,2)
            
            if score < best_score:
                best_score, best_gid = score, gid

        if best_gid is None:
            continue

        assigned[best_gid] = d
        unassigned.remove(best_gid)
        r = row_by_id[best_gid]
        district_pop[d] += int(r[POP_COL])
        district_dem[d] += int(r[DEM_COL])
        district_rep[d] += int(r[REP_COL])
        district_precincts[d] += 1
        frontiers[d].discard(best_gid)
        for nbr in adj[best_gid]:
            if nbr not in assigned:
                frontiers[d].add(nbr)

        iter += 1
        if iter % 50 == 0:
            print(f"Iteration {iter} | Expanding district {d+1} | Current pop: {district_pop[d]:,} | Unassigned left: {len(unassigned)}")

    
    
    initial_assignment_dict = {gid: d + 1 for gid, d in assigned.items()}

   
   # Hill climbing algorithm to optimize the population, keeping it between the range of 0.95 to 1.05, this is
   # just if the popualtion is outside the ranges it would try to flip the berder districts to get a proper result
    
    MAX_BALANCE_ITERATIONS = 50000

    # uses the assignment dict to get the population value 
    def _bal_get_pop_dict(assignment_dict):
        populations = {i: 0 for i in range(1, K + 1)}
        try:
            for geoid, district in assignment_dict.items():
                if district in populations:
                    populations[district] += row_by_id[geoid][POP_COL]
        except NameError:
            return None
        return populations

    #this helper function is to help with finding border precincts of districts that are overpopulated/
    #underpopulated which can flip to underpopulated district
    def _bal_find_flip(assignment_dict, district_populations):
        over_districts = {d for d, pop in district_populations.items() if pop > MAX_POP}
        under_districts = {d for d, pop in district_populations.items() if pop < MIN_POP}
        
        if not over_districts and not under_districts:
            return None 

        target_precincts = []
        if over_districts:
            target_precincts = [p for p, d in assignment_dict.items() if d in over_districts]
        else:
            in_bounds_districts = {d for d, pop in district_populations.items() if MIN_POP <= pop <= MAX_POP}
            target_precincts = [p for p, d in assignment_dict.items() if d in in_bounds_districts]

        if not target_precincts:
            return None 

        random.shuffle(target_precincts)

        for precinct in target_precincts:
            if not G.has_node(precinct): continue
            old_dist = assignment_dict[precinct]
            neighbors = list(G.neighbors(precinct))
            for n in neighbors:
                if n not in assignment_dict: continue
                new_dist = assignment_dict[n]
                if new_dist == old_dist: continue
                is_good_flip = False
                if old_dist in over_districts and (new_dist in under_districts or new_dist not in over_districts):
                    is_good_flip = True
                elif new_dist in under_districts and old_dist not in under_districts:
                    is_good_flip = True
                if is_good_flip:
                    return (precinct, old_dist, new_dist)
        return None 

    # actual hill climbing algorithm which looks at random border precincts and tries to check if they would balance the 
    # population while keeping contiguity
    def balance_population_hill_climb(initial_assignment_dict, max_iterations):
        current_assignment = initial_assignment_dict.copy()
        accepted_flips = 0
        
        for i in range(max_iterations):
            district_pops = _bal_get_pop_dict(current_assignment)
            if district_pops is None: return None
            
            are_balanced = True
            for d, pop in district_pops.items():
                if not (MIN_POP <= pop <= MAX_POP):
                    are_balanced = False
                    break
            
            if are_balanced:
                print(f"Success! Population balanced in {i} iterations.")
                return current_assignment
                
            flip_to_try = _bal_find_flip(current_assignment, district_pops)
            
            if flip_to_try is None:
                print(f"Stuck, Could not find a valid flip at iteration {i}.")
                return current_assignment

            precinct, old_dist, new_dist = flip_to_try
            temp_assignment = current_assignment.copy()
            temp_assignment[precinct] = new_dist
            
            temp_df = pd.DataFrame(temp_assignment.items(), columns=['GEOID20', 'District'])
            
            if isDistrictContiguous(old_dist, temp_df, nj_contiguity):
                current_assignment = temp_assignment
                accepted_flips += 1
                if accepted_flips % 1 == 0:
                     print(f"Hill-Climb Iter {i}:  ACCEPTED: (Total Flips: {accepted_flips})")
            
        print(f"Failed to balance population after {max_iterations} iterations.")
        return current_assignment

    final_assignment_dict = balance_population_hill_climb(
        initial_assignment_dict,
        MAX_BALANCE_ITERATIONS
    )

    assign_df = pd.DataFrame({
        "GEOID20": list(final_assignment_dict.keys()),
        "district": [final_assignment_dict[g] for g in final_assignment_dict.keys()]
    })

    final_district_pop, final_district_dem, final_district_rep = [], [], []
    for d in range(1, K + 1):
        district_geoids = assign_df[assign_df['district'] == d]['GEOID20']
        district_data = nj_precinct_data[nj_precinct_data['GEOID20'].isin(district_geoids)]
        final_district_pop.append(district_data[POP_COL].sum())
        final_district_dem.append(district_data[DEM_COL].sum())
        final_district_rep.append(district_data[REP_COL].sum())

    summary = pd.DataFrame({
        "district": list(range(1, K + 1)),
        "population": final_district_pop,
        "dem_votes_2020_pres": final_district_dem,
        "rep_votes_2020_pres": final_district_rep
    })
    summary["rep_margin"] = summary["rep_votes_2020_pres"] - summary["dem_votes_2020_pres"]
    summary["lean"] = np.where(summary["rep_margin"] > 0, "R", "D")

    return assign_df, summary

In [ ]:
# Cell 6: create fair and unfair maps
#fair_map, fair_summary = build_map_balanced(objective="fair")
unfair_map, unfair_summary = build_map_balanced(objective="unfair", advantage="REP")

#fair_map 
unfair_map

Iteration 50 | Expanding district 9 | Current pop: 84,013 | Unassigned left: 6299
Iteration 100 | Expanding district 9 | Current pop: 150,578 | Unassigned left: 6249
Iteration 150 | Expanding district 9 | Current pop: 201,158 | Unassigned left: 6199
Iteration 200 | Expanding district 9 | Current pop: 236,967 | Unassigned left: 6149
Iteration 250 | Expanding district 9 | Current pop: 260,757 | Unassigned left: 6099
Iteration 300 | Expanding district 10 | Current pop: 109,525 | Unassigned left: 6049
Iteration 350 | Expanding district 9 | Current pop: 313,305 | Unassigned left: 5999
Iteration 400 | Expanding district 12 | Current pop: 77,344 | Unassigned left: 5949
Iteration 450 | Expanding district 10 | Current pop: 198,873 | Unassigned left: 5899
Iteration 500 | Expanding district 9 | Current pop: 373,953 | Unassigned left: 5849
Iteration 550 | Expanding district 9 | Current pop: 421,868 | Unassigned left: 5799
Iteration 600 | Expanding district 12 | Current pop: 137,417 | Unassigned le

In [ ]:
# Cell 7: export outputs
#fair_map.to_csv('fair_map_GG.csv', index=False)
unfair_map.to_csv('unfair_map_GG.csv', index=False)
#fair_summary.to_csv('fair_map_summary.csv', index=False)
unfair_summary.to_csv('unfair_map_summary.csv', index=False)

'Files exported: fair_map_GROUP.csv, unfair_map_GROUP.csv, fair_map_summary.csv, unfair_map_summary.csv'


In [ ]:
# Cell 8: quick estimate of seat advantage based on presidential vote margins
fair_R = int((fair_summary['rep_margin'] > 0).sum())
#unfair_R = int((unfair_summary['rep_margin'] > 0).sum())
"""{
    "fair_map_est_R_seats": fair_R,
    "unfair_map_est_R_seats": unfair_R,
    "target_pop_per_district": target
}"""
